# I. Fitting Data (Python)
Este notebook resuelve la Parte I del trabajo, simulando los datos y entrenando distintas redes neuronales con `scikit-learn`.
Las imágenes se guardan con sufijo `_python` para diferenciarlas de las versiones en R y Julia.

In [ ]:
# Importación de librerías
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
# Configuración de semilla para reproducibilidad
np.random.seed(42)

In [ ]:
# Simulación de datos
n = 300
x = np.linspace(0, 2 * np.pi, n)
epsilon = np.random.normal(0, 0.1, n)
y = np.sin(x) + epsilon

plt.figure(figsize=(8,4))
plt.scatter(x, y, s=15, color='blue', label='Datos simulados')
plt.title('Simulación de datos: y = sin(x) + ε')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.tight_layout()
plt.savefig('simulacion_python.png', dpi=300)
plt.show()

In [ ]:
# Entrenamiento de redes neuronales con distintas funciones de activación
X = x.reshape(-1, 1)
y_true = y
activations = ['logistic', 'tanh', 'relu']
models = {}
errors = {}

for act in activations:
    print(f'Entrenando red con activación: {act}')
    model = MLPRegressor(hidden_layer_sizes=(50,50,50), activation=act, solver='adam', max_iter=3000, random_state=42)
    model.fit(X, y_true)
    models[act] = model
    y_pred = model.predict(X)
    errors[act] = mean_squared_error(y_true, y_pred)

In [ ]:
# Gráfico comparativo de activaciones
x_grid = np.linspace(0, 2*np.pi, 500).reshape(-1,1)
plt.figure(figsize=(10,6))
plt.scatter(x, y, s=15, color='gray', label='Datos reales')
for act in activations:
    y_pred = models[act].predict(x_grid)
    plt.plot(x_grid, y_pred, label=f'NN ({act})', linewidth=2)
plt.title('Ajuste de NNs con diferentes funciones de activación')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.tight_layout()
plt.savefig('comparacion_activaciones_python.png', dpi=300)
plt.show()

In [ ]:
# Red neuronal mixta (relu -> tanh -> logistic)
m1 = MLPRegressor(hidden_layer_sizes=(50,), activation='relu', solver='adam', max_iter=3000, random_state=42)
m1.fit(X, y_true)
y_stage1 = m1.predict(X).reshape(-1,1)

m2 = MLPRegressor(hidden_layer_sizes=(50,), activation='tanh', solver='adam', max_iter=3000, random_state=42)
m2.fit(y_stage1, y_true)
y_stage2 = m2.predict(y_stage1).reshape(-1,1)

m3 = MLPRegressor(hidden_layer_sizes=(50,), activation='logistic', solver='adam', max_iter=3000, random_state=42)
m3.fit(y_stage2, y_true)
y_pred_mix = m3.predict(y_stage2)

In [ ]:
# Gráfico de comparación incluyendo la red mixta
plt.figure(figsize=(10,6))
plt.scatter(x, y, s=15, color='gray', label='Datos reales')
for act in activations:
    plt.plot(x_grid, models[act].predict(x_grid), label=f'NN ({act})')
plt.plot(x, y_pred_mix, '--', color='black', linewidth=2, label='NN (relu-tanh-logistic)')
plt.title('Comparación: NNs con distintas funciones de activación')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.tight_layout()
plt.savefig('red_mixta_python.png', dpi=300)
plt.show()

In [ ]:
# Evaluación del desempeño de cada red (MSE)
print('Errores (MSE) obtenidos:')
for act, err in errors.items():
    print(f'{act}: {err:.5f}')

err_mix = mean_squared_error(y_true, y_pred_mix)
print(f'relu-tanh-logistic: {err_mix:.5f}')

best = min(errors, key=errors.get)
print(f'La red con activación "{best}" tuvo el mejor ajuste global (menor MSE).')

In [ ]:
# Guardar tabla con los resultados
df_errors = pd.DataFrame({
    'Activacion': list(errors.keys()) + ['relu-tanh-logistic'],
    'MSE': list(errors.values()) + [err_mix]
})
df_errors.to_csv('resultados_mse_python.csv', index=False)
print('Resultados guardados en resultados_mse_python.csv')